In [78]:
#!pip install deltalake
#!pip install duckdb

In [77]:
Nbr_Files_to_Download   = 280
schema                  = 'aemo'
summary_table           = 'summary'
storage_options         = {"allow_unsafe_rename":"true"}

**<mark>Import</mark>**

In [79]:
import duckdb
import pyarrow as pa
import pyarrow.compute as pc
import os
from   deltalake import DeltaTable
from   deltalake.writer import write_deltalake, try_get_deltatable
import glob
import re
import requests
from   shutil import unpack_archive
from   urllib.request import urlopen
import multiprocessing

**<mark>Download Some Data from the web</mark>**

In [80]:
def download(url,Path,Onelake_table,total_files):
    if not os.path.exists(Path):
      os.makedirs(Path, exist_ok=True)
    result = urlopen(url).read().decode('utf-8')
    pattern = re.compile(r'[\w.]*.zip')
    filelist1 = pattern.findall(result)
    filelist_unique = dict.fromkeys(filelist1)
    filelist =sorted(filelist_unique)
    dt =try_get_deltatable(Onelake_table,storage_options=storage_options)
    if dt is not None:
     existing_files = dt.to_pandas(columns=["file"])['file'].unique().tolist()
     existing_files = [w.replace('.CSV','.zip') for w in existing_files]
    else:
     existing_files =[]
    files_to_upload = list(set(filelist) - set(existing_files))
    files_to_upload = sorted(list(dict.fromkeys(files_to_upload)))[:total_files]
    print(str(len(files_to_upload)) + ' New File Downloaded')
    if len(files_to_upload) != 0 :
      for x in files_to_upload:
           with requests.get(url+x, stream=True) as resp:
            if resp.ok:
              with open(f"{Path}{x}", "wb") as f:
               for chunk in resp.iter_content(chunk_size=4096):
                f.write(chunk)
    return files_to_upload

**<mark>Unzip</mark>**

In [81]:
def uncompress(x):
        unpack_archive(str(Zip_Path+x), str(Csv_Path), 'zip')
def unzip(Source, Destination,files_to_upload):
    if not os.path.exists(Destination):
      os.makedirs(Destination, exist_ok=True)
    print(str(len(files_to_upload)) + ' New File uncompressed')
    if len(files_to_upload) != 0 :
      with multiprocessing.Pool() as pool:
       for _ in pool.imap_unordered(uncompress, files_to_upload, chunksize=1):
         pass
      return [Csv_Path + i for i in files_to_upload]
    else:
     return []

**<mark>Price Today </mark>**

In [82]:
def get_price(files_to_upload_full_Path):
  raw =duckdb.sql(f"""from read_csv({files_to_upload_full_Path},
  Skip=1,header =0,all_varchar=1,
  columns={{
  'I' : 'VARCHAR', 'DISPATCH' : 'VARCHAR', 'PRICE' : 'VARCHAR', 'xx' : 'VARCHAR', 'SETTLEMENTDATE' : 'VARCHAR', 'RUNNO' : 'VARCHAR', 'REGIONID' : 'VARCHAR',
   'DISPATCHINTERVAL' : 'VARCHAR', 'INTERVENTION' : 'VARCHAR', 'RRP' : 'VARCHAR', 'EEP' : 'VARCHAR', 'ROP' : 'VARCHAR', 'APCFLAG' : 'VARCHAR',
    'MARKETSUSPENDEDFLAG' : 'VARCHAR', 'LASTCHANGED' : 'VARCHAR', 'RAISE6SECRRP' : 'VARCHAR', 'RAISE6SECROP' : 'VARCHAR', 'RAISE6SECAPCFLAG' : 'VARCHAR',
    'RAISE60SECRRP' : 'VARCHAR', 'RAISE60SECROP' : 'VARCHAR', 'RAISE60SECAPCFLAG' : 'VARCHAR', 'RAISE5MINRRP' : 'VARCHAR', 'RAISE5MINROP' : 'VARCHAR',
    'RAISE5MINAPCFLAG' : 'VARCHAR', 'RAISEREGRRP' : 'VARCHAR', 'RAISEREGROP' : 'VARCHAR', 'RAISEREGAPCFLAG' : 'VARCHAR', 'LOWER6SECRRP' : 'VARCHAR',
     'LOWER6SECROP' : 'VARCHAR', 'LOWER6SECAPCFLAG' : 'VARCHAR', 'LOWER60SECRRP' : 'VARCHAR', 'LOWER60SECROP' : 'VARCHAR', 'LOWER60SECAPCFLAG' : 'VARCHAR',
     'LOWER5MINRRP' : 'VARCHAR', 'LOWER5MINROP' : 'VARCHAR', 'LOWER5MINAPCFLAG' : 'VARCHAR', 'LOWERREGRRP' : 'VARCHAR', 'LOWERREGROP' : 'VARCHAR',
      'LOWERREGAPCFLAG' : 'VARCHAR', 'PRICE_STATUS' : 'VARCHAR', 'PRE_AP_ENERGY_PRICE' : 'VARCHAR', 'PRE_AP_RAISE6_PRICE' : 'VARCHAR', 'PRE_AP_RAISE60_PRICE' : 'VARCHAR',
       'PRE_AP_RAISE5MIN_PRICE' : 'VARCHAR', 'PRE_AP_RAISEREG_PRICE' : 'VARCHAR', 'PRE_AP_LOWER6_PRICE' : 'VARCHAR', 'PRE_AP_LOWER60_PRICE' : 'VARCHAR',
        'PRE_AP_LOWER5MIN_PRICE' : 'VARCHAR', 'PRE_AP_LOWERREG_PRICE' : 'VARCHAR', 'RAISE1SECRRP' : 'VARCHAR', 'RAISE1SECROP' : 'VARCHAR', 'RAISE1SECAPCFLAG' : 'VARCHAR',
         'LOWER1SECRRP' : 'VARCHAR', 'LOWER1SECROP' : 'VARCHAR', 'LOWER1SECAPCFLAG' : 'VARCHAR', 'PRE_AP_RAISE1_PRICE' : 'VARCHAR',
          'PRE_AP_LOWER1_PRICE' : 'VARCHAR', 'CUMUL_PRE_AP_ENERGY_PRICE' : 'VARCHAR', 'CUMUL_PRE_AP_RAISE6_PRICE' : 'VARCHAR', 'CUMUL_PRE_AP_RAISE60_PRICE' : 'VARCHAR',
          'CUMUL_PRE_AP_RAISE5MIN_PRICE' : 'VARCHAR', 'CUMUL_PRE_AP_RAISEREG_PRICE' : 'VARCHAR', 'CUMUL_PRE_AP_LOWER6_PRICE' : 'VARCHAR',
          'CUMUL_PRE_AP_LOWER60_PRICE' : 'VARCHAR', 'CUMUL_PRE_AP_LOWER5MIN_PRICE' : 'VARCHAR', 'CUMUL_PRE_AP_LOWERREG_PRICE' : 'VARCHAR',
          'CUMUL_PRE_AP_RAISE1_PRICE' : 'VARCHAR', 'CUMUL_PRE_AP_LOWER1_PRICE' : 'VARCHAR', 'OCD_STATUS' : 'VARCHAR', 'MII_STATUS' : 'VARCHAR',
  }},
  filename =1,null_padding = true,ignore_errors=1,auto_detect=false)
  where I='D' and PRICE ='PRICE'

                  """)
  columns = list(set(raw.columns) - {'SETTLEMENTDATE','REGIONID','I','PRICE','filename','OCD_STATUS','MII_STATUS','DISPATCH','PRICE_STATUS','LASTCHANGED'})

  exprs = [
    duckdb.ColumnExpression(x).cast(duckdb.typing.DOUBLE).alias(x)
    for x in columns
  ]
  rel2 = raw.select('SETTLEMENTDATE','REGIONID','I','PRICE','filename','OCD_STATUS','MII_STATUS','DISPATCH','PRICE_STATUS','LASTCHANGED',*exprs)
  final=duckdb.sql(""" select *exclude(SETTLEMENTDATE,I,xx,'PRICE','filename'),
  cast (SETTLEMENTDATE as TIMESTAMPTZ) as SETTLEMENTDATE,
  cast(SETTLEMENTDATE as date) as date,
  parse_filename(filename) as file,
  0 as PRIORITY,
  isoyear (cast (SETTLEMENTDATE as TIMESTAMPTZ)) as YEAR
  from rel2  """)
  return final.arrow()

**<mark>SCADA Today </mark>**

In [83]:
# @title
def get_scada(files_to_upload_full_Path):
  raw =duckdb.sql(f"""from read_csv({files_to_upload_full_Path},
  Skip=1,header =0,all_varchar=1,
  columns={{
  'I' : 'VARCHAR', 'DISPATCH' : 'VARCHAR', 'UNIT_SCADA' : 'VARCHAR', 'xx' : 'VARCHAR', 'SETTLEMENTDATE' : 'timestamp', 'DUID' : 'VARCHAR', 'SCADAVALUE' : 'double','LASTCHANGED' : 'timestamp'
  }},
  filename =1,null_padding = true,ignore_errors=1,auto_detect=false)
  where I='D' and SCADAVALUE !=0
                  """)
  scada=duckdb.sql(""" select  DUID,SCADAVALUE as INITIALMW, cast(0 as double ) as INTERVENTION,
   cast (SETTLEMENTDATE as TIMESTAMPTZ) as SETTLEMENTDATE,
   cast(SETTLEMENTDATE as date) as date,
   parse_filename(filename) as file,
   0 as PRIORITY ,
   isoyear (cast (SETTLEMENTDATE as timestamp)) as YEAR
   from raw
    """)
  return scada.arrow()

**<mark>DUID </mark>**

In [84]:
def get_duid(Onelake_table,storage_options):
    DUID_Path = "/lakehouse/default/Files/0_Source/Dimensions/DUID/"
    import pathlib
    import requests
    pathlib.Path(DUID_Path).mkdir(parents=True, exist_ok=True)
    url = "https://www.aemo.com.au/-/media/Files/Electricity/NEM/Participant_Information/NEM-Registration-and-Exemption-List.xls"
    s = requests.Session()
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'}
    r = s.get(url,headers=headers)
    r.content
    output = open(DUID_Path+"NEM-Registration-and-Exemption-List.xls", 'wb')
    output.write(r.content)
    output.close()
    duckdb.sql(f"""
    INSTALL spatial;
    LOAD spatial;
    create or replace table DUID as
    SELECT Region,DUID,first("Fuel Source - Descriptor") as FuelSourceDescriptor,first(Participant) as Participant
    FROM st_read('{DUID_Path}NEM-Registration-and-Exemption-List.xls', layer = 'PU and Scheduled Loads',open_options = ['HEADERS=FORCE'])
    group by all
    """)

    import requests
    dls = "https://data.wa.aemo.com.au/datafiles/post-facilities/facilities.csv"
    resp = requests.get(dls)
    output = open(DUID_Path+"facilities_WA.csv", 'wb')
    output.write(resp.content)
    output.close()

    duckdb.sql(f""" select 'WA1' as Region  , "Facility Code" as DUID ,"Participant Name" as Participant from read_csv_auto('{DUID_Path}facilities_WA.csv')""").to_view('x')

    duckdb.sql("""select x.Region,x.DUID, TECHNOLOGY as FuelSourceDescriptor,Participant from x
    left join (select * FROM read_csv_auto('https://github.com/djouallah/aemo_fabric/raw/main/WA_ENERGY.csv',header=1)) as z
    on x.duid=z.duid """).to_view('DUID_WA')

    duckdb.sql("""
    create or replace table states(RegionID varchar, States varchar) ;
    insert into states values
    ('WA1' , 'Western Australia') ,
    ('QLD1' , 'Queensland')  ,
    ('NSW1' , 'New South Walles')  ,
    ('TAS1' , 'Tasmania')  ,
    ('SA1' , 'South Australia')  ,
    ('VIC1' , 'Victoria')
    """)

    df =duckdb.sql(f""" with xx as (select * from DUID union BY NAME select * from DUID_WA)
                select States,trim(DUID) as DUID,min(Region) as Region, min(FuelSourceDescriptor) as FuelSourceDescriptor,
                min(Participant) as Participant
                from xx
                JOIN states on xx.Region = states.RegionID
                group by all
                """).arrow()
    write_deltalake(Onelake_table, df,mode="overwrite",engine='rust',storage_options = storage_options)

_**<mark>Summary</mark>**_

In [85]:
sch = pa.schema([("settlementdate", pa.timestamp("us","UTC")),
                    ("date", pa.date32()),
                    ("DUID", pa.string()),
                    ("States", pa.string()),
                    ("mw", pa.float64()),
                    ("price", pa.float64()),
         ])
DeltaTable.create(f'/lakehouse/default/Tables/{schema}/{summary_table}', schema=sch,mode="ignore",storage_options = storage_options)

DeltaTable()

In [86]:
sch = pa.schema([("table", pa.string()),("version", pa.int64())])
DeltaTable.create(f'/lakehouse/default/Tables/{schema}/metadata', schema=sch,mode="ignore",storage_options = storage_options)


DeltaTable()

In [87]:
def full_refresh(schema):
    return f"""
      CREATE OR REPLACE VIEW {schema}.scada AS SELECT * FROM delta_scan('/lakehouse/default/Tables/{schema}/scada');
      CREATE OR REPLACE VIEW {schema}.price AS SELECT * FROM delta_scan('/lakehouse/default/Tables/{schema}/price');
      select
        cast(s.settlementdate as TIMESTAMPTZ) as settlementdate,
        s.date,
        s.DUID,
        States,
        max(s.INITIALMW) as mw,
        max(p.RRP ) as price
      from ( select * from  {schema}.scada  where INTERVENTION = 0 and INITIALMW <> 0)  s
            LEFT JOIN {schema}.duid d ON s.DUID = d.DUID
            LEFT JOIN ( select * from {schema}.price where INTERVENTION = 0)  p ON s.SETTLEMENTDATE = p.SETTLEMENTDATE AND d.Region = p.REGIONID
      group by
        all
      order by s.date
    """

In [88]:
def Incremental_refresh(schema,max_timestamp):
    return f"""
      select
        cast(s.settlementdate as TIMESTAMPTZ) as settlementdate,
        s.date,
        s.DUID,
        States,
        max(s.INITIALMW) as mw,
        max(p.RRP ) as price
      from
        {schema}.scada_today  s
        JOIN {schema}.duid d ON s.DUID = d.DUID
        JOIN {schema}.price_today        p ON s.SETTLEMENTDATE = p.SETTLEMENTDATE AND d.Region = p.REGIONID
      where
        s.INTERVENTION = 0
        and p.INTERVENTION = 0
        and INITIALMW <> 0
        and s.settlementdate > '{max_timestamp}' and p.settlementdate > '{max_timestamp}'
      group by
        all
      order by s.date
    """

In [89]:
def get_max():
    # duckdb don't like scanning table with empty rows, fixed in 1.1.3
    dt = DeltaTable(f'/lakehouse/default/Tables/{schema}/{summary_table}',storage_options=storage_options)
    dataset = dt.to_pyarrow_dataset().to_table(columns=["settlementdate"])
    max_timestamp = pc.max(dataset['settlementdate']).as_py()
    if  not max_timestamp :
        max_timestamp = '1900-01-01'
    else :
        max_timestamp =  max_timestamp.strftime("%Y-%m-%d %H:%M:%S")
    return max_timestamp

# Process Data

In [90]:
Web_Path          = "http://nemweb.com.au/Reports/Current/DispatchIS_Reports/"
Zip_Path          = "/lakehouse/default/Files/0_Source/Current/DispatchIS_Reports/"
Csv_Path          =  "/tmp/"
Onelake_table     = '/lakehouse/default/Tables/'+schema+'/price_today'

files_to_upload = download(Web_Path,Zip_Path,Onelake_table,Nbr_Files_to_Download)
files_to_upload_full_Path_zip=unzip(Zip_Path,Csv_Path,files_to_upload)
files_to_upload_full_Path = [w.replace('.zip','.CSV') for w in files_to_upload_full_Path_zip]

if len(files_to_upload_full_Path) >0 :
  df = get_price(files_to_upload_full_Path)
  write_deltalake(Onelake_table, df,mode="append",partition_by=['date'],engine='rust',storage_options=storage_options)
else:
  print('all loaded already')


0 New File Downloaded
0 New File uncompressed
all loaded already


In [91]:
Web_Path          = "http://nemweb.com.au/Reports/Current/Dispatch_SCADA/"
Zip_Path          = "/lakehouse/default/Files/0_Source/Current/Dispatch_SCADA/"
Csv_Path          =  "/tmp/scada/"
Onelake_table     = '/lakehouse/default/Tables/'+schema+'/scada_today'

files_to_upload = download(Web_Path,Zip_Path,Onelake_table,Nbr_Files_to_Download)
files_to_upload_full_Path_zip=unzip(Zip_Path,Csv_Path,files_to_upload)
files_to_upload_full_Path = [w.replace('.zip','.CSV') for w in files_to_upload_full_Path_zip]
if len(files_to_upload_full_Path) >0 :
  df = get_scada(files_to_upload_full_Path)
  write_deltalake(Onelake_table, df,mode="append",partition_by=['date'],engine='rust',storage_options=storage_options)
else:
  print('all loaded already')

0 New File Downloaded
0 New File uncompressed
all loaded already


In [92]:
Onelake_table = f'/lakehouse/default/Tables/{schema}/duid'
dt =try_get_deltatable(Onelake_table,storage_options=storage_options)
if dt is None:
    get_duid(Onelake_table,storage_options)
else:
    print('duid loaded already')

duid loaded already


In [93]:
Onelake_table = f'/lakehouse/default/Tables/{schema}/mstdatetime'
dt =try_get_deltatable(Onelake_table,storage_options=storage_options)
if dt is None:
    df=duckdb.sql(""" SELECT cast(unnest(generate_series(cast ('2018-04-01' as date), cast('2024-12-31' as date), interval 5 minute)) as TIMESTAMPTZ) as SETTLEMENTDATE,
            strftime(SETTLEMENTDATE, '%I:%M:%S %p') as time,
            cast(SETTLEMENTDATE as date ) as date,
            EXTRACT(year from date) as year,
            EXTRACT(month from date) as month
            """).arrow()
    write_deltalake(Onelake_table, df,mode="ignore",storage_options = storage_options,engine='rust')
else:
    print('mstdatetime loaded already')

mstdatetime loaded already


In [94]:
Onelake_table = f'/lakehouse/default/Tables/{schema}/calendar'
dt =try_get_deltatable(Onelake_table,storage_options=storage_options)
if dt is None:
    df=duckdb.sql(""" SELECT cast(unnest(generate_series(cast ('2018-04-01' as date), cast('2024-12-31' as date), interval 1 day)) as date) as date,
            EXTRACT(year from date) as year,
            EXTRACT(month from date) as month
            """).arrow()
    write_deltalake(Onelake_table, df,mode="ignore",storage_options = storage_options,engine='rust')
else:
    print('calendar loaded already')

calendar loaded already


In [95]:
#### check if scada was updated
duckdb.sql(f"create SCHEMA IF NOT EXISTS {schema}")
for tbl in ['scada_today','duid','price_today',summary_table]:
    duckdb.sql(f"CREATE OR REPLACE VIEW {schema}.{tbl} AS SELECT * FROM delta_scan('/lakehouse/default/Tables/{schema}/{tbl}');")
dt = try_get_deltatable(f'/lakehouse/default/Tables/{schema}/scada',storage_options=storage_options)
if dt is None:
 current_version = -1
else:
 current_version = dt.version()
Previous_version = duckdb.sql(f"SELECT max(version) FROM delta_scan('/lakehouse/default/Tables/{schema}/metadata');").fetchone()[0]
Previous_version = Previous_version or 0
print(f'scada current  version {current_version}')
print(f'scada previous version {Previous_version}')
if current_version != Previous_version and current_version !=-1:
    print('scada table was updated, trigger full refresh')
    get_duid(f'/lakehouse/default/Tables/{schema}/duid',storage_options)
    df = duckdb.sql(full_refresh(schema)).arrow()
    r = df.num_rows
    if r >0 :
        RG=8_000_000
        write_deltalake('/lakehouse/default/Tables/'+schema+"/"+summary_table,
                         df,
                         mode="overwrite",
                         max_rows_per_group = RG,
                         min_rows_per_group = RG,
                         storage_options= storage_options,
                         engine='rust')
        print('store new version')
        df = pa.table({'table': ['scada'], 'version': [current_version]})
        write_deltalake(f'/lakehouse/default/Tables/{schema}/metadata', df,mode="overwrite",engine='rust',storage_options = storage_options)
else:
    print('scada table was not updated, trigger incremental refresh')
    max_timestamp = get_max()
    df = duckdb.sql(Incremental_refresh(schema,max_timestamp)).arrow()
    r = df.num_rows
    if r >0 :
        write_deltalake('/lakehouse/default/Tables/'+schema+"/"+summary_table, df, mode="append", storage_options= storage_options, engine='rust')
        print(f'new {r} rows inserted')
    else :
        print("no new data")

scada current  version -1
scada previous version 0
scada table was not updated, trigger incremental refresh
no new data
